In [53]:
# Imports section
from itertools import combinations

In [54]:
# Take Parameters Input Section
print("Enter the number of transactions: ")
n = int(input())
transactions = []
print("Enter minimum support value: ")
min_support = int(input())
print("Enter minimum confidence value between 0 to 1: ")
min_confidence = float(input())

Enter the number of transactions: 
5
Enter minimum support value: 
3
Enter minimum confidence value between 0 to 1: 
0.8


In [55]:
# Take Transaction set as input
i = 0
while i < n:
    print("Enter items for transaction: " + str(i+1))
    items = input()
    transactions.append(items.split(" "))
    i = i+1
i = 0

Enter items for transaction: 1
A B C
Enter items for transaction: 2
B D E
Enter items for transaction: 3
B A C
Enter items for transaction: 4
A C B
Enter items for transaction: 5
A C D B


In [56]:
# Generate initial candidate list
candidate_list = {}
for items_list in transactions:
    i = 0
    for item in items_list:
        if item in candidate_list.keys():
            items_list_sub = items_list[0:i]
            if item not in items_list_sub:
                candidate_list[item] = candidate_list[item] + 1
        else:
            candidate_list[item] = 1
        i = i+1

print("Candidate List: ")
print(candidate_list)

Candidate List: 
{'A': 4, 'B': 5, 'C': 4, 'D': 2, 'E': 1}


In [57]:
# Eliminate from candidate list where support count < minimum support
candidate_list = {key:value for key,value in candidate_list.items() if value >= min_support}

print("Candidate List After Elimination: ")
print(candidate_list)

Candidate List After Elimination: 
{'A': 4, 'B': 5, 'C': 4}


In [58]:
"""
Utility function that takes in a list with items and support
and returns list containing items where support >= minimum support

:param param_candidate_list:
:return:
"""
def eliminate(param_candidate_list):
    param_candidate_list = {key:value for key,value in param_candidate_list.items() if value >= min_support}
    return param_candidate_list

# Start Apriori
flag = True
new_candidate_list = {}
prev_candidate_list = {}
count = 2
# archive contains the support values of all iterations. Useful for calculating association rules confidence.
archive = candidate_list.copy()

while True:
    new_keys = []
    prev_candidate_list = new_candidate_list.copy()
    new_candidate_list = {}
    new_keys = tuple(list(combinations(candidate_list, count)))
    
    # Form new keys and update support
    for new_key in new_keys:
        for items_list in transactions:
            if set(new_key).issubset(set(items_list)) and new_key in new_candidate_list.keys() and new_key in archive.keys():
                new_candidate_list[new_key] = new_candidate_list[new_key] + 1
                archive[new_key] = archive[new_key] + 1
            elif set(new_key).issubset(set(items_list)):
                new_candidate_list[new_key] = 1
                archive[new_key] = 1
    
    # Print current candidate list
    print("Item list at the end of iteration: ")
    print(new_candidate_list)
    new_candidate_list = eliminate(new_candidate_list)
    # Eliminate items having support less than minimum
    print("Item list after elimination: ")
    print(new_candidate_list)
    candidate_list = set()
    for key_list in new_candidate_list.keys():
        for item in key_list:
            candidate_list.add(item)
    
    # BREAK CONDITION: Break if all items eliminated from candidate list
    if len(new_candidate_list) == 0:
        print("All items eliminated. BREAK.")
        new_candidate_list = prev_candidate_list.copy()
        break
    
    # Continue to next iteration
    count = count+1
    print("-------------------------------------------------------------------------------")

# Print frequent item set
print("-------------------- FREQUENT ITEM SET --------------------")
print(new_candidate_list)

Item list at the end of iteration: 
{('A', 'B'): 4, ('A', 'C'): 4, ('B', 'C'): 4}
Item list after elimination: 
{('A', 'B'): 4, ('A', 'C'): 4, ('B', 'C'): 4}
-------------------------------------------------------------------------------
Item list at the end of iteration: 
{('C', 'A', 'B'): 4}
Item list after elimination: 
{('C', 'A', 'B'): 4}
-------------------------------------------------------------------------------
Item list at the end of iteration: 
{}
Item list after elimination: 
{}
All items eliminated. BREAK.
-------------------- FREQUENT ITEM SET --------------------
{('C', 'A', 'B'): 4}


In [60]:
# Start Mining Association Rules
for freq_item_set in new_candidate_list.keys():
    rules = {}
    print("-------------------- ASSOCIATION RULES FOR: " + str(freq_item_set) + " --------------------")
    keys=[]
    
    # Form the rules here and calculate confidence for each.
    for i in range(0,len(freq_item_set)):
        key = [freq_item_set[i]]
        key.append(tuple([x for j,x in enumerate(freq_item_set) if j != i]))
        keys.append(key)
        try:
            confidence = round(archive[freq_item_set]/archive[key[0]],2)
        except KeyError:
            confidence = round(archive[freq_item_set]/archive[key[0][::-1]],2)
        rules[tuple(key)] = confidence
    
    # For the same keys formed above, reverse and find confidence again.
    for key in keys:
        x = key[::-1]
        try:
            confidence = round(archive[freq_item_set]/archive[x[0]],2)
        except KeyError:
            confidence = round(archive[freq_item_set]/archive[x[0][::-1]],2)
        rules[tuple(x)] = confidence
    
    # Print association rules, along with confidence and classification -> whether strong or weak.
    for key in rules.keys():
        if rules[key] >= min_confidence:
            print(str(key) + "; CONFIDENCE: " + str(rules[key]) + "; STRONG")
        else:
            print(str(key) + "; CONFIDENCE: " + str(rules[key]) + "; WEAK")
    
    print("")

-------------------- ASSOCIATION RULES FOR: ('C', 'A', 'B') --------------------
('C', ('A', 'B')); CONFIDENCE: 1.0; STRONG
('A', ('C', 'B')); CONFIDENCE: 1.0; STRONG
('B', ('C', 'A')); CONFIDENCE: 0.8; STRONG
(('A', 'B'), 'C'); CONFIDENCE: 1.0; STRONG
(('C', 'B'), 'A'); CONFIDENCE: 1.0; STRONG
(('C', 'A'), 'B'); CONFIDENCE: 1.0; STRONG

